In [ ]:
import ee
import os
import pandas as pd
from datetime import datetime, timedelta
from src.controllers.time_series_hls import HLS
from src.controllers.sentinel import Sentinel2Processor
from src.controllers.geometry import GeoDataFrameProcessor
from src.controllers.eras import PrecipitationTemperatureRadiationData

In [2]:
# Authenticate and initialize Google Earth Engine
ee.Authenticate()
ee.Initialize()

In [ ]:
# Analysis dates
START_DATE = '2023-01-01'
END_DATE = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')

# Path to geometry file
GEOMETRY_PATH = r"C:\Users\ander\OneDrive\Área de Trabalho\msu\bases\base\fields_tests.geojson"

PRODUCT = 's2'

# Initialize geometry processor
geometry_processor = GeoDataFrameProcessor(GEOMETRY_PATH)

In [4]:
def update_start_dates(file_path, default_start_date):
    """Update start dates for NDVI and weather data based on existing files."""
    ndvi_file = os.path.join(file_path, "ndvi_data.csv")
    weather_file = os.path.join(file_path, "weather_data.csv")

    if os.path.exists(ndvi_file):
        ndvi_df = pd.read_csv(ndvi_file)
        ndvi_start_date = pd.to_datetime(ndvi_df.sort_values(by='date')['date'].values[-1]).strftime('%Y-%m-%d')
    else:
        ndvi_start_date = default_start_date

    if os.path.exists(weather_file):
        weather_df = pd.read_csv(weather_file)
        weather_start_date = pd.to_datetime(weather_df.sort_values(by='date')['date'].values[-1]).strftime('%Y-%m-%d')
    else:
        weather_start_date = default_start_date

    return weather_start_date, ndvi_start_date

In [5]:
def data_base_process(POLYGON_ID):

    # Create directory to save DataFrames
    directory = f"data_/{POLYGON_ID}"

    # Process each individual polygon
    vertices, geometry = geometry_processor.extract_coordinates(POLYGON_ID)

    # Update start dates for NDVI and weather data
    weather_start_date, ndvi_start_date = update_start_dates(directory, START_DATE)

    # Get NDVI data based on product type
    if PRODUCT == 'hls':
        satellite = HLS(geometry, ndvi_start_date, END_DATE)
        ndvi_df = satellite.convert_to_dataframe()
    else:
        s2_processor = Sentinel2Processor(ndvi_start_date, END_DATE, geometry, cloud_threshold=0)
        ndvi_df = s2_processor.get_filtered_df()

    # Extract latitude and longitude
    latitude = geometry.getInfo()['coordinates'][0][0][1]
    longitude = geometry.getInfo()['coordinates'][0][0][0]

    # Get weather data: precipitation, temperature, and radiation
    weather_data = PrecipitationTemperatureRadiationData(latitude, longitude, weather_start_date, END_DATE)
    weather_df = weather_data.get_dataframe()

    # Save or update NDVI and weather data
    if not os.path.exists(directory):
        os.makedirs(directory)
        ndvi_df.to_csv(os.path.join(directory, "ndvi_data.csv"), index=False)
        weather_df.to_csv(os.path.join(directory, "weather_data.csv"), index=False)
    else:
        # Update NDVI data
        ndvi_history_df = pd.read_csv(os.path.join(directory, "ndvi_data.csv"))
        ndvi_combined_df = pd.concat([ndvi_history_df, ndvi_df], ignore_index=True).drop_duplicates(subset=['date'])
        ndvi_combined_df.to_csv(os.path.join(directory, "ndvi_data.csv"), index=False)

        # Update weather data
        weather_history_df = pd.read_csv(os.path.join(directory, "weather_data.csv"))
        weather_combined_df = pd.concat([weather_history_df, weather_df], ignore_index=True).drop_duplicates(subset=['date'])
        weather_combined_df.to_csv(os.path.join(directory, "weather_data.csv"), index=False)

    print(f'Polygon {POLYGON_ID} finished.')



In [ ]:
for POLYGON_ID in geometry_processor.index_polygons:
    data_base_process(POLYGON_ID)